# Libraries

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import display
from difflib import context_diff
from pprint import pprint
import sqlalchemy
from datetime import datetime as dt
from os import getcwd
import seaborn as sns
import random
import time
import operator
import re

# Functions and classes

In [ ]:
####################
# Helper functions #
####################

def splitbynewlines(tosplit):
    remr = []
    toret = []
    if isinstance(tosplit, basestring):
        remn = [tosplit]
    else:
        remn = tosplit
    for elem in remn:
        remr = remr + elem.split('\n')
    for elem in remr:
        toret = toret + elem.split('\r')
    return toret

def dedup(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def catcols(df):
    catcols = [c for c in df.select_dtypes(include=['object']).columns] + \
            [c for c in df.columns if c[-2:].lower() == 'yn'] + \
            [c for c in df.columns if c[-2:].lower() == 'id'] + \
            [c for c in df.columns[df.isin([0,1]).all()]]
    # Remove duplicates
    return dedup(catcols)

def sumsumsqdiff(x,df):
    toret = 0
    for i, row in df.iterrows():
        toret = toret + ((x - row)**2).sum()
    return toret

###########
# Classes #
###########

class FindAssumptionBreakers:
    def __init__(self,df,identifier):

        if identifier not in df.columns:
            raise KeyError("Specified unique identifier '{}' is not a column of the passed dataframe.".format(identifier))
        
        self.df = df
        
        # Results that will be returned:
        self.res = pd.DataFrame(self.df[identifier]).copy()
        
        # A place for applied functions to store errors, overriding each time, to avoid an error per row
        self.errors = ''
        
        # How many example rows have been shown so far?
        self.eg = 0
        
        # For the max/min range tests:
        self.ltgt = (operator.lt,operator.gt)
        
        # Identify columns by dtype:
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        dates = ['<M8', 'datetime64']
        numericcols = [c for c in self.df.select_dtypes(include=numerics).columns]
        datecols = [c for c in self.df.select_dtypes(include=dates).columns]
        bothcolgroups = numericcols + datecols
        
        # Create parameters:
        numdict = zip(numericcols,[(0,1000)] * len(numericcols))
        datedict = zip(datecols,[('2002-01-01',dt.now().date().isoformat())] * len(datecols))
        self.ranges = dict(numdict + datedict)
        
        # Need these for later
        self.pattern = re.compile('|'.join(bothcolgroups))
        self.cannull = []
        self.tests = []
        
        # Are these new parameterS?
        self.newparams = False
    
    def parameterTemplate(self):
        m = []
        m.append('cannull = [\n')
        # Print columns commented out, to allow easy additition to cannull list:
        cols = []
        for col in self.df.columns:
            comment = ("#" if not self.df[col].isnull().all() else "")
            cols += [comment + "'" + col + "'"]
        m.append(',\n'.join(cols))
        m.append('\n]\n\nranges = ' + str(self.ranges).replace('),','),\n'))
        # Example tests
        m.append("\n\ntests = [#'A <= B',\n#'A + B * 2 = C'\n]")
        print(''.join(m))
        
    def TESTparameterTemplate(self):
        m = []
        m.append('cannull = [\n')
        # Print columns commented out, to allow easy additition to cannull list:
        m.append(',\n'.join(["#'" + x + "'" for x in self.df.columns]))
        m.append('\n]\n\nranges = ')
        ranges = str(self.ranges)
        ranges = ranges.replace('{','{\n')
        ranges = ranges.replace('}','\n}')
        ranges = ranges.replace('),','),\n')
        for line in ranges.split('\n'):
            if "id':" in line:
                m.append('# ')
            elif "yn':" in line:
                m.append('# ')
            m.append(line + '\n')
        # Example tests
        m.append("\n\ntests = [#'A <= B',\n#'A + B * 2 = C'\n]")
        print(''.join(m))
    
    def updateParameters(self,cannull=None,ranges=None,tests=None):
        if cannull:
            self.cannull = cannull
        if tests:
            self.tests = tests
        if ranges:
            self.ranges = ranges
        if any([cannull,tests,ranges]):
            self.newparams = True
        
    def inc(self,series,msg):
        # Increment the count of broken invariants for rows that broke this invariant
        ones = series * 1
        self.res['CNT'] = self.res['CNT'] + ones
        # Add the error message to the messages field
        msgs = series.replace(True,msg + '; ').replace(False,'')
        self.res['MSGS'] = self.res['MSGS'].str.cat(msgs)
        
    def nullTest(self,col):
        if col not in self.cannull and self.df[col].isnull().values.any(): # For columns that aren't ok to be null, but have a null,
            self.inc(self.df[col].isnull(),'nullTest: ' + col) # Add one to the variant count for each null row
    
    def rangeTest(self,col):
        if col in self.ranges.keys(): # For each column that we know the 'valid' range of
            for comp in [0,1]: # Check if less than min or greater than max,
                self.inc(self.ltgt[comp](self.df[col],self.ranges[col][comp]),'rangeTest: ' + col) # And add 1 to each row where true

    def evalTest(self):
        for t in self.tests:
            self.inc(self.df.apply(self.testRow,axis=1,test=t),'evalTest: ' + t)
            
    def testRow(self,row,test):
        toeval = self.pattern.sub(lambda x: "row['" + x.group() + "']", test)
        try:
            evalres = not eval(toeval)
        except:
            self.errors = 'Invalid Test: ' + test + ' --> ' + toeval + '\n'
            evalres = False
        return evalres
    
    def testassumptions(self):
        # Initialize:
        self.res['CNT'] = pd.Series(0, index=self.res.index)
        self.res['MSGS'] = pd.Series('', index=self.res.index)
        
        # Run tests
        for col in self.df.columns:
            self.nullTest(col)
            self.rangeTest(col)
        self.evalTest()
        
        # Record state
        if self.newparams:
            self.eg = 0
        self.newparams = False
            
    def assumptionbreaker(self):
        # Sort the data by the number of invariants broken, descending:
        merged = pd.merge(self.res,self.df).sort_values('CNT',ascending=False)
        # Get the first row not yet displayed
        todisp = merged.iloc[self.eg]
        # Update self.eg to be the index of examples not yet seen
        self.eg = self.eg + 1
        if todisp['CNT'] == 0:
            print('No assumptions are broken.')
        else:
            # Display the relevant rows
            print('This row broke {} assumptions:\n{}'.format(todisp['CNT'],todisp['MSGS']))
            display(todisp[[x for x in todisp.index if x not in ['CNT','MSGS']]].T)
            
            
    def showErrors(self):
        print(self.errors)
            
class FindUnusualExamples:
    def __init__(self,df,exampleid):
        
        if exampleid not in df.columns:
            raise KeyError("Specified unique identifier '{}' is not a column of the passed dataframe.".format(exampleid))
        
        self.df = df
        
        # Results that will be returned:
        self.res = pd.DataFrame(self.df[exampleid]).copy()
        
        # Example ID
        self.egid = exampleid
    
    def catDeviScore(self,col):
        # Find the % of the data in each category
        self.res[col] = self.df[col].values
        dist = pd.value_counts(self.df[col].values,normalize=True)
        if dist.empty:
            return
        # We will give categories with the highest frequency a score of 0, and
        # the categories with the lowest frequency a score of 1
        dist2 = dedup(dist.values)
        dist3 = [dist2[0] / x if x != 0 else 0 for x in dist2]
        dist4 = [x - 1 for x in dist3]
        dist5 = [x / dist4[-1] if dist4[-1] != 0 else 0 for x in dist4]
        dist6 = pd.DataFrame(dist2,dist5).reset_index()
        # Mapping series
        mapser = pd.merge(pd.DataFrame(dist).reset_index(),dist6,on=0)
        mapdict = mapser[['index_x','index_y']].set_index('index_x').to_dict()['index_y']
        self.res[col] = self.res[col].map(mapdict)
        
    def contDeviScore(self,theseries,col):
        percen = theseries.map(lambda x: stats.percentileofscore(theseries,x,kind='mean'))
        self.res[col] = percen.map(lambda x: 2 * abs(50 - x) / 100)
#         for k in ['rank','weak','strict','mean']:
#             display(theseries.map(lambda x: stats.percentileofscore(theseries,x,kind=k)))
    
    def numDeviScore(self,col):
        self.contDeviScore(self.df[col],col)
    
    def dateDeviScore(self,col):
        seconds = self.df[col].map(lambda x: (x - dt.fromtimestamp(0)).total_seconds())
        self.contDeviScore(seconds,col)
    
    def parameterTemplate(self):
#         yns = [c for c in self.df.columns if c[-2:].lower() == 'yn']
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        dates = ['<M8', 'datetime64']
        typedict = {}
        catcols = catcols(self.df)
        typedict['categorical'] = catcols
        numcols = [c for c in self.df.select_dtypes(include=numerics).columns if c not in catcols]
        typedict['numbers'] = numcols
        datecols = [c for c in self.df.select_dtypes(include=dates).columns if c not in catcols]
        typedict['dates'] = datecols
        nullcols = [x for x in self.df.columns if self.df[x].isnull().all()]
#         typedict['categorical'] = yns + [c for c in self.df.select_dtypes(include=['object']).columns if c not in (datecols + numcols + nullcols)]
        
        print('parameters = ' + str(typedict).replace(',',',\n'))
    
    def initialize(self,typedict):
        if 'dates' in typedict:
            for col in typedict['dates']:
                self.dateDeviScore(col)
        if 'numbers' in typedict:
            for col in typedict['numbers']:
                self.numDeviScore(col)
        if 'categorical' in typedict:
            for col in typedict['categorical']:
                self.catDeviScore(col)
    
    def example(self):
        percentiles = [x for x in self.res.columns if x not in [self.egid,'Score']]
        
        if not hasattr(self, 'prevvec'):
            self.prevvec = pd.DataFrame(0,index=[0],columns=percentiles)
        
        # Add sqrt(sum(squared(differences))) to score for that example
        self.res['Score'] = self.res.apply(sumsumsqdiff,axis=1,args=(self.prevvec,))
        
        # Index of row with the highest score:
        highid = self.res['Score'].idxmax()
        
        # Example ID of the first row with the highest score:
        weirdest = self.res.loc[highid][self.egid]
        
        # Display row from the original dataframe identified by that exampleid
        rowtoshow = self.df.loc[self.df[self.egid] == weirdest].T
        rowtoshow.columns = ['Values']*len(rowtoshow.columns)
        
        # And show the uniqueness scores so it's clear why the example was chosen
        rowtoshow['Uniqueness'] = self.res.loc[highid].drop(['uniqueid','Score'])
        rowtoshow['Uniqueness'] = rowtoshow['Uniqueness'].fillna('N/A')
        
        print('The following row is the most unique, out of the rows you have not yet reviewed:')
        display(rowtoshow)
        print('The uniqueness scores are 0 for median values and the most common categories, and 1 is for mins, maxs, and the least common categories.')
        
        # Add that example to the previous vector array
        self.prevvec = self.prevvec.append(self.res.loc[highid][percentiles])

# Input parameters

Enter the name of your query file, a context you want to run on, the number of examples you want, and the name of the column that contains a unique identifier for each row:

In [ ]:
print('You are currently in ' + getcwd())

In [ ]:
# What is the name of your query file?
queryfile = 'EXAMPLE.sql'

# How many examples do you need, minimum?
numegs = 200

# How many days do you want to run the query over at first?
initialdays = 1

# What is the name of the column contains a unique identifier?
# WARNING! If you put something here that's not truly unique, it will cause subtle errors later
uniqueid = 'id'

### Connect to the database

Get your database handle, dbh, here

In [ ]:
# Put the code here you need to get your database handle

# Difference from the last query

What were your most recent changes to your query?

In [ ]:
if 'oldquery' in locals():
    oldoldquery = oldquery # Back up the backup

if 'query' in locals():
    oldquery = query # Back up the present

# Open the query
with open(queryfile, 'r') as f:
    query = f.read()

if 'oldoldquery' in locals() and query == oldquery: # If there was no change,
    oldquery = oldoldquery # the diff before that will be more interesting

# Show what last changed about the query
if 'oldquery' in locals() and 'query' in locals():
    for line in context_diff(splitbynewlines(oldquery),splitbynewlines(query)):
        pprint(line)

# Run the query

In [ ]:
starttime = time.time()

if ':days' not in friendlyq: # If there is no day span specified, just run it
    try:
        df = pd.read_sql(friendlyq,dbh)
    except Exception as exception:
        print(exception.orig)
        print(exception.statement)
    numrows = len(df)
    print('Finished: Pulled {} rows, in {:.2f} seconds.'.format(numrows,time.time()-starttime))
else: # If there is a dayspan, run it over increasingly larger dayspans until you get enough rows
    daysspan = initialdays
    print('Running query over {:.2f} days'.format(daysspan))
    try:
        df = pd.read_sql(friendlyq,dbh,params={'days':daysspan})
    except Exception as exception:
        print(exception.orig)
        print(exception.statement)
    numrows = len(df)

    # Did we get enough examples?
    while numrows < numegs:
        try:
            # How far from the number of rows we want are we?
            faroff = float(numegs) / numrows
        except ZeroDivisionError:
            faroff = 2

        # Let's increase the search, but not go up by more than two, to be careful not to overadjust
        daysspan *= min(faroff * 1.1,2)

        if daysspan > 365:
            print("Stopping, on the assumption you don't want to run this over more than a year.")
            break

        print('Only got {} rows. Now running query over {:.2f} days'.format(numrows,daysspan))
        df = pd.read_sql(friendlyq,dbh,params={'days':daysspan})
        numrows = len(df)

    print('Finished: Pulled {} rows, from {:.2f} days, in {:.2f} seconds.'.format(numrows,daysspan,time.time()-starttime))

# Take a high level look at the data

### Categorical variables

In [ ]:
# Show me the top N values in each category:
topN = 5

nullcols = []
for x in catcols(df):
    todisp = pd.value_counts(df[x].values,normalize=True).iloc[:topN]
    if not todisp.empty:
        display(pd.DataFrame(todisp.rename(str(x)).map(lambda x: "{0:.4f} %".format(x * 100))))
        print('Top {} represent {:.1%} of rows.\n\n'.format(topN,todisp.sum()))
    else:
        nullcols = nullcols + [str(x)]
if nullcols:
    print('The following categorical columns are entirely null:\n')
    for x in nullcols:
        print(x)

### Continuous variables

In [ ]:
desc = pd.DataFrame()
for col in df:
    s = df[col]
    if col in catcols(df):
        continue
    if s.dtype not in ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']:
        continue
    d = {}
    d['count'] = s.count()
    d['sum'] = s.sum()
    d['mean'] = s.mean()
    numnull = s.isnull().sum() 
    d['%null'] = float(numnull) / (numnull + d['count'])
    d['min'] = s.min()
    d['10%'] = s.dropna().quantile(0.1)
    d['50%'] = s.dropna().quantile(0.5)
    d['90%'] = s.dropna().quantile(0.9)
    d['max'] = s.max()
    desc[col] = pd.Series(d)
    
def readableNumbers(x):
    toins = ''
    if isinstance(x, float):
        decimalplaces = 3
        if x.is_integer():
            decimalplaces = 0
        toins = '.' + str(decimalplaces) + 'f'
    return ('{:,' + toins + '}').format(x)

colorder = ['count','sum','mean','%null','min','10%','50%','90%','max']
display(desc.applymap(readableNumbers).T[colorder].style.applymap(lambda x: 'text-align:right'))

sns.pairplot(df[desc.columns].dropna())

# Test your assumptions about the data 
Copy the output of the next cell into the next cell, and then modify it.

#### For the cannull variable:
Remove the comment '#' from any column that can be null

#### For the ranges variable:
The default is to assume every number is between 0 and 1000 inclusive, and every date is between 2002 and 2100. Change those to the range you would be shocked to see those columns fall outside of.

#### For the tests variable:
Write out equations--in the same format as the examples--that you would assume are always true. Use column names instead of 'A', 'B' and 'C'

In [ ]:
instance = FindAssumptionBreakers(df,uniqueid)
instance.parameterTemplate()

## Put your updated parameters here

In [ ]:
cannull = [
#'id',
#'event',
#'eventdate',
#'username',
#'uniqueid',
'deleted',
'deletedby',
#'created',
#'createdby',
'lastmodified',
'lastmodifiedby',
#'claimid',
'claimnoteid',
'transactionid',
'kickreasonid'
]

ranges = {
#     u'kickreasonid': (0, 1000),
 u'created': ('2017-01-01', '2017-10-06'),
#  u'claimid': (0, 1000),
#  u'id': (0, 1000),
#  u'claimnoteid': (0, 1000),
 u'eventdate': ('2017-01-01', '2017-10-06')}

tests = [
    'eventdate <= created',
#'A + B * 2 = C'
]

## Feed the updated parameters to the instance of the class

In [ ]:
instance.updateParameters(cannull,ranges,tests)
# Test all of the assumptions
instance.testassumptions()

## Run this to see a row that breaks your assumptions

In [ ]:
# Display a row that broke the assumptions
instance.assumptionbreaker()

# Find unusual examples

In [ ]:
instance = FindUnusualExamples(df,uniqueid)
instance.parameterTemplate()

### Remove any list items, or even full dictionary keys, that you don't want to be part of the unusualness tests.

In [ ]:
parameters = {'dates': [u'eventdate',
 u'created'],
 'categorical': [u'event',
 u'username',
 u'createdby']}

In [ ]:
# For some reason stats.percentileofscore() throws an error if it's not imported right before being called:
from scipy import stats
instance.initialize(parameters)

In [ ]:
instance.example()
print('\nContextid: {}'.format(contextid))

# Drop this data to CSV

In [ ]:
csvname = queryfile[:-3] + 'csv'

print('The CSV, by default will be "{}".\n'.format(csvname))
print('To change this, uncomment and change "example path" below')

In [ ]:
csvname = "Examplepath.csv"

df.to_csv(csvname,index=False)